# Data Preparation

In [1]:
from os.path import join as opjoin
import pandas as pd
import numpy as np
from PIL import Image
from pathlib import Path
import os

In [2]:
#dataset_path = '/mnt/fred/reddit/database_1597063213/'
dataset_path = 'H://reddit/corgis/'
df = pd.read_csv(opjoin(dataset_path, 'submissions_.csv'))
print('{} submissions loaded...'.format(len(df)))

4967 submissions loaded...


In [3]:
df.head()

,author,created_utc,id,is_video,num_comments,permalink,score,title,url,flagged,bboxX,bboxY,bboxSize
0,si102dk,1595628805,hxb6s4,False,26,/r/corgi/comments/hxb6s4/most_boopable_snout_e...,545,Most boopable snout ever look for my corgi,https://i.redd.it/mvtcj8gtpvc51.png,False,0.0,3.0,750.0
1,BrujaMala13,1595592713,hx0non,False,31,/r/corgi/comments/hx0non/please_welcome_scarle...,800,Please Welcome Scarlet Pumpernickel 🥰,https://i.redd.it/q1balemtqsc51.jpg,False,0.0,209.0,1020.0
2,ricardotacopaco,1595528615,hwkzd8,False,24,/r/corgi/comments/hwkzd8/bring_your_loaf_to_wo...,195,Bring your loaf to work day was a success,https://i.redd.it/4panlr58gnc51.jpg,False,208.0,174.0,2004.0
3,jafiswert,1595523557,hwjc74,False,18,/r/corgi/comments/hwjc74/ellie_our_latest_addi...,724,"Ellie, our latest addition to our family. She ...",https://i.redd.it/qjdg2im61nc51.jpg,False,0.0,244.0,3024.0
4,dycbaylor02,1595522754,hwj2yi,False,18,/r/corgi/comments/hwj2yi/waiting_on_a_snack/,885,Waiting on a snack,https://i.redd.it/i8ind3osymc51.jpg,False,72.0,771.0,2624.0


In [21]:
print(len(df[~pd.isnull(df['bboxX'])].index.tolist()))

3280


In [4]:
df_clean = df[df['flagged'] == False]

Path(opjoin(dataset_path, 'images', '256')).mkdir(exist_ok=True)
Path(opjoin(dataset_path, 'images', '512')).mkdir(exist_ok=True)
Path(opjoin(dataset_path, 'images', '1024')).mkdir(exist_ok=True)

flagged = []
for i, x in df_clean.iterrows():
    imagepath = opjoin(dataset_path, 'images', x['id'])
    if os.path.isfile(imagepath + '.jpeg'):
        imagepath += '.jpeg'
    elif os.path.isfile(imagepath + '.png'):
        imagepath += '.png'
    else:
        continue
        
    try:
        im = Image.open(imagepath)
        im = im.convert('RGB')
    except:
        continue
        
    if im.size[0] < 700 or im.size[1] < 700:
        flagged.append(True)
        continue
        
    a = x['bboxX']
    b = x['bboxY']
    c = x['bboxSize']
    
    #im_cropped = im.crop((a, b, a+c, b+c))
    #for size in [256, 512, 1024]:
    #    im_resized = im_cropped.resize((size, size))
    #    im_resized.save(opjoin(dataset_path, 'images', str(size), x['id']+'.jpg'), 'JPEG')
        
    flagged.append(False)

In [16]:
flagged = pd.Series(flagged, index=df_clean.index)
df_clean = df_clean[flagged.values == False]

[False False False ... False  True  True]


In [17]:
df_clean.to_csv(opjoin(dataset_path, 'submissions_clean.csv'), index=False)

# Mask RCNN

In [18]:
import torch
import torchvision
torch.cuda.is_available()

ModuleNotFoundError: No module named 'torch'